In [89]:
from abc import ABC, abstractmethod
import numpy as np
import pandas as pd
import requests
import asyncio

class Model(ABC):
    @abstractmethod
    def download(self, categories):
     pass

    @abstractmethod
    def transform_data(self, raw_data):
     pass

class WoysaLoader(Model):

       def __init__(self):
        self.base_url = 'https://analitika.woysa.club/images/panel/json/download/niches.php'

        async  def download_data(self, category, skip):
           url = f"{self.base_url}?skip={skip}&price_min=0&price_max=1060225&up_vy_min=0&up_vy_max=108682515&up_vy_pr_min=0&up_vy_pr_max=29006&sum_min=10006&sum_max=82432725&feedbacks_min=0&feedbacks_max=32767&trend=false&sort=sum_sale&sort_dir=-1&id_cat={category}"
           data = requests.get(url)
           return pd.read_excel(data.content)

        async def get_data_from_category(category):
            df_result=pd.DataFrame()
            for skip in range(0, 200, 100):
              print (category, skip)
              df = await download_data(category, skip)
              if df.empty: break
              if df_result.empty: df_result = df
              else: df_result = pd.concat ([df_result,df])
            return df_result

        async def get_data_from_categories(batch_categories):
            df_result = pd.DataFrame()
            for category in batch_categories:
             df = await get_data_from_category(category)
             if df_result.empty: df_result = df
             else: df_result = pd.concat ([df_result,df])
            return df_result

        if __name__ == "__main__":
          categories = np.array_split(range(100, 120), 10)
          df_result = asyncio.get_event_loop().run_until_complete(asyncio.gather(*list(map(lambda x: asyncio.ensure_future(get_data_from_categories(x)), categories))))
          print(df_result)